## PyTorch Tutorial 10 - Dataset Transforms

https://www.youtube.com/watch?v=X_QOZEko5uE

In [1]:
import torch
import torchvision
import numpy as np

In [2]:
# dataset = torchvision.datasets.MNIST(root="./data", download=True)

In [3]:
dataset = torchvision.datasets.MNIST(root="./data", 
                                     transform = torchvision.transforms.ToTensor())

To know about transforms - refer here: https://pytorch.org/vision/stable/transforms.html

In [4]:
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
from torch.utils.data import Dataset, DataLoader

## code from last tutorial
class WineDataset_old(Dataset):
    def __init__(self):
        # data loading
        xy = np.loadtxt("./data/wine.csv", skiprows=1, delimiter=",", dtype=np.float32)
        self.x = torch.from_numpy(xy[:,1:]) # all except first
        self.y = torch.from_numpy(xy[:, 0]) # firs col
        self.n_samples = xy.shape[0]
        
    def __getitem__(self, index):
        # datatset[0]
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.n_samples
        
dataset = WineDataset_old()


In [22]:
# Lets extend above

## code from last tutorial
class WineDataset(Dataset):
    def __init__(self, transform=None):
        # data loading
        xy = np.loadtxt("./data/wine.csv", skiprows=1, delimiter=",", dtype=np.float32)
        self.x = xy[:,1:] # all except first
        self.y = xy[:, [0]] # first col, we put [0] as we want it to be column matrix
        self.n_samples = xy.shape[0]
        
        self.transform = transform
        
    def __getitem__(self, index):
        # datatset[0]
        sample =  self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
        
    def __len__(self):
        return self.n_samples

# A custom trasform
class ToTensor:
    def __call__(self, sample):  #This is callable object
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

# Another custom transform - a multiply trasform
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
        
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target


In [23]:
dataset = WineDataset(transform = ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])


In [24]:
# We can put a list of transforms
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform = composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features)

<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])


We see above that MulTransform made input multiply by 2